# Key classification with an RNN for Tensor Flow Lite

Classify USB data which comes in packets of 8 bytes representing shift and key presses.

## Install

In [0]:
!pip install tensorflow
!pip install numpy

## Setup input pipeline

Data for the pipeline needs to be in the form of a numpy array so we wrap our simple array in the call to np.array

Todo: Generate this data some how.

In [0]:
import tensorflow as tf
import numpy as np

In [0]:
dataset = np.array([ [2,0,4,0,0,0,0,0],[2,0,4,0,0,0,0,0],[0,0,5,0,0,0,0,0]])  # The letter A with a shift
resultset = np.array([[1,0],[1,0],[0,1]])

for ex in dataset:
  print(ex)

## Prepare the data for training

Split the data 80:20 so we've got some data to validate our model against.

In [0]:
from sklearn.model_selection import train_test_split

train_dataset, test_dataset, train_results, test_results = train_test_split(dataset, resultset, test_size = 0.2)

Lets take a look at one of these to see how it now looks. The shape of the model is important as it needs to mirror the input shape of the first layer of the model.

In [0]:
print("Batch shape:", train_dataset.shape)
print("label shape:", train_results.shape)

## Create the model

Build a machine learning model using the Keras API which simplifies this rocess.

The layers represent nodes in a recurrent neural network (RNN). This processes sequence input by iterating through the elements. RNNs pass the outputs from one timestep to their input—and then to the next.

It is important to include the input_shape parameter so that the convertion process can work correctly.

The output has 2 nodes representing a Happy Comment and an Angry Comment. If both are unset it is a neutral comment and both set is an ambigious comment.

In [0]:
model = tf.keras.Sequential([
  tf.keras.layers.Dense(16, activation='relu', input_shape=(8,)),
  tf.keras.layers.Dense(8)
  tf.keras.layers.Dense(2)
  ])

model.summary()

The model needs to be compiled before use. The several named defaults are used here. There are many choices for the optimiser and this will affect both the training duration and the size of the end mode.

In [0]:
model.compile (optimizer='rmsprop',
              loss='mse',
              metrics=['accuracy'])


## Train the model

The model is trained on a set of known data and then validated against a typically smaller set of known data.

In [0]:
history = model.fit(train_dataset,train_results, epochs=1,
                    validation_data=(test_dataset,test_results), 
                    validation_steps=1)

In [0]:
test_loss, test_acc = model.evaluate(test_dataset)

print('Test Loss: {}'.format(test_loss))
print('Test Accuracy: {}'.format(test_acc))

In [0]:
# Save the model
model.save("keyclassification_model") 

## Test the model

In [0]:
# Do some prediction based on the inputs

res = model.predict(np.array([[2, 0, 4, 0, 0, 0, 0, 0],
                              [0, 0, 0, 0, 0, 0, 0, 0],
                              [2, 0, 5, 6, 0, 0, 0, 0],
                              [0,0,5,0,0,0,0,0]]))
print(res)


# Export the model

Convert the model to TFLite then format as a big C array.

https://github.com/eloquentarduino/tinymlgen

In [0]:
!pip install tinymlgen

In [0]:
from tinymlgen import port

c_code = port(model,optimize=True,pretty_print=True)

print(len(c_code))

File size needs to be small enough both to fit into the flash memory but also to run in RAM. So look for sizes in the low KBs. Once we've converted our model we can save it out to disk.

In [0]:
c_file = open(r"text_model.h","w+")

n = c_file.write(c_code)
c_file.close()

# Testing the TFLite model

The easiest library to get TFLite working on your Arduino is [EloquentTinyML](https://github.com/eloquentarduino/EloquentTinyML)

Copy the header file into your arduino project and include it as follows. Note that to work correctly the variable needs to be declared before the setup and main functions.

```
#undef min //Needed for TensorFlow lite to correctly redeclare the min and max macros.
#undef max
#include <EloquentTinyML.h>

#include "text_model.h"

#define NUMBER_OF_INPUTS 8
#define NUMBER_OF_OUTPUTS 2
// you may need to tweek this value: it's a trial and error process
#define TENSOR_ARENA_SIZE 2*1024

Eloquent::TinyML::TfLite<NUMBER_OF_INPUTS, NUMBER_OF_OUTPUTS, TENSOR_ARENA_SIZE> ml((unsigned char*)model_data);
```


